In [1]:
# create spark session
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("Introduction to Spark")
    .master("local[*]")
    .getOrCreate()
)
spark

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/07/14 16:12:57 WARN Utils: Your hostname, DESKTOP-DMC84B8, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/07/14 16:12:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/14 16:12:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/07/14 16:12:59 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
# create sample data
emp_data = [
    ["001","101","John Doe","30","Male","50000","2015-01-01"],
    ["002","101","Jane Smith","25","Female","45000","2016-02-15"],
    ["003","102","Bob Brown","35","Male","55000","2014-05-01"],
    ["004","102","Alice Lee","28","Female","48000","2017-09-30"],
    ["005","103","Jack Chan","40","Male","60000","2013-04-01"],
    ["006","103","Jill Wong","32","Female","52000","2018-07-01"],
    ["007","101","James Johnson","42","Male","70000","2012-03-15"],
    ["008","102","Kate Kim","29","Female","51000","2019-10-01"],
    ["009","103","Tom Tan","33","Male","58000","2016-06-01"],
    ["010","104","Lisa Lee","27","Female","47000","2018-08-01"],
    ["011","104","David Park","38","Male","65000","2015-11-01"],
    ["012","105","Susan Chen","31","Female","54000","2017-02-15"],
    ["013","106","Brian Kim","45","Binary","75000","2011-07-01"],
    ["014","107","Emily Lee","26","Female","46000","2019-01-01"],
    ["015","106","Michael Lee","37","Male","63000","2014-09-30"],
    ["016","107","Kelly Zhang","30","Female","49000","2018-04-01"],
    ["017","105","George Wang","34","Male","57000","2016-03-15"],
    ["018","104","Nancy Liu","29","Mechanic","50000","2017-06-01"],
    ["019","103","Steven Chen","36","Male","62000","2015-08-01"],
    ["020","102","Grace Kim","32","Female","53000","2018-11-01"]
]

emp_schema = "employee_id string, department_id string, name string, age string, gender string, salary string, hire_date string"

In [6]:
emp = spark.createDataFrame(data=emp_data, schema=emp_schema)
emp.show()

+-----------+-------------+-------------+---+--------+------+----------+
|employee_id|department_id|         name|age|  gender|salary| hire_date|
+-----------+-------------+-------------+---+--------+------+----------+
|        001|          101|     John Doe| 30|    Male| 50000|2015-01-01|
|        002|          101|   Jane Smith| 25|  Female| 45000|2016-02-15|
|        003|          102|    Bob Brown| 35|    Male| 55000|2014-05-01|
|        004|          102|    Alice Lee| 28|  Female| 48000|2017-09-30|
|        005|          103|    Jack Chan| 40|    Male| 60000|2013-04-01|
|        006|          103|    Jill Wong| 32|  Female| 52000|2018-07-01|
|        007|          101|James Johnson| 42|    Male| 70000|2012-03-15|
|        008|          102|     Kate Kim| 29|  Female| 51000|2019-10-01|
|        009|          103|      Tom Tan| 33|    Male| 58000|2016-06-01|
|        010|          104|     Lisa Lee| 27|  Female| 47000|2018-08-01|
|        011|          104|   David Park| 38|    Ma

In [7]:
# case when
# apply on a gender attribute: 'M' if male, 'F' if female and null otherwise
from pyspark.sql.functions import when, col

emp_gender_fixed = emp.withColumn("gender_fixed", when(col("gender") == "Male", "M")
                                  .when(col("gender") == "Female", "F")
                                  .otherwise(None))
emp_gender_fixed.show()

+-----------+-------------+-------------+---+--------+------+----------+------------+
|employee_id|department_id|         name|age|  gender|salary| hire_date|gender_fixed|
+-----------+-------------+-------------+---+--------+------+----------+------------+
|        001|          101|     John Doe| 30|    Male| 50000|2015-01-01|           M|
|        002|          101|   Jane Smith| 25|  Female| 45000|2016-02-15|           F|
|        003|          102|    Bob Brown| 35|    Male| 55000|2014-05-01|           M|
|        004|          102|    Alice Lee| 28|  Female| 48000|2017-09-30|           F|
|        005|          103|    Jack Chan| 40|    Male| 60000|2013-04-01|           M|
|        006|          103|    Jill Wong| 32|  Female| 52000|2018-07-01|           F|
|        007|          101|James Johnson| 42|    Male| 70000|2012-03-15|           M|
|        008|          102|     Kate Kim| 29|  Female| 51000|2019-10-01|           F|
|        009|          103|      Tom Tan| 33|    Male|

In [17]:
## antherway using expr
from pyspark.sql.functions import expr

emp_gender_fixed_1 = emp.withColumn("gender_fixed", expr("case when gender='Male' then 'M' when 'gender'='Female' then 'F' else null"))
emp_gender_fixed_1.show()

ParseException: 
[PARSE_SYNTAX_ERROR] Syntax error at or near 'gender'. SQLSTATE: 42601 (line 1, pos 10)

== SQL ==
case when gender='Male' then 'M' when 'gender'='Female' then 'F' else null
----------^^^
